# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from imageio import imread
import datetime
import os
from pathlib import Path
from skimage.transform import resize
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
import math
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10 

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,3,7,10,14,17,20,23,26,29]
    x = 10 #frames
    y = 90 #width of resized image
    z = 90 #height of resized image
    while True:
        t = np.random.permutation(folder_list)
        num_batches = math.floor(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                     
                    if image.shape[1] == 160:
                        # converting this image to 1:1 aspect ratio, then resizing
                        image = resize(image[:,20:140,:],(y,z)).astype(np.float32)
                    else:
                        image = resize(image,(y,z)).astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] = [[float(i)/255 for i in row] for row in image[:,:,0]]
                    batch_data[folder,idx,:,:,1] = [[float(i)/255 for i in row] for row in image[:,:,1]]
                    batch_data[folder,idx,:,:,2] = [[float(i)/255 for i in row] for row in image[:,:,2]]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        if len(t)%batch_size !=0:
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                     
                    if image.shape[1] == 160:
                        # converting this image to 1:1 aspect ratio, then resizing
                        image = resize(image[:,20:140,:],(y,z)).astype(np.float32)
                    else:
                        image = resize(image,(y,z)).astype(np.float32)
                        
                    batch_data[folder,idx,:,:,0] = [[float(i)/255 for i in row] for row in image[:,:,0]]
                    batch_data[folder,idx,:,:,1] = [[float(i)/255 for i in row] for row in image[:,:,1]]
                    batch_data[folder,idx,:,:,2] = [[float(i)/255 for i in row] for row in image[:,:,2]]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels 


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

### CNN+RNN

In [6]:
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Input, GRU, LSTM, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D
from keras.layers.convolutional import Conv3D, MaxPooling3D, Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras import initializers

In [7]:
dropout = 0.5
weights = initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=2)
rnn_output_size = 128
nb_classes = 5
img_rows, img_cols = 90, 90
img_channels = 3
frames = 10

In [8]:
def get_tranferlearning_timedistributed_model():
    
    # Get base model: VGG16
    video = Input(shape=(frames,img_rows,img_cols,img_channels))
    cnn_base = VGG16(input_shape=(img_rows,img_cols,img_channels),weights="imagenet",include_top=False)
    
    cnn_out = GlobalAveragePooling2D()(cnn_base.output)
    cnn = Model(input=cnn_base.input, output=cnn_out)
    cnn.trainable = True
    encoded_frames = TimeDistributed(cnn)(video)
    # Flat the layer  
    hidden_layer = TimeDistributed(Flatten())(encoded_frames)   
    encoded_sequence = GRU(256,return_sequences=False, dropout = dropout)(hidden_layer)
    outputs = Dense(output_dim=nb_classes, activation="softmax")(encoded_sequence)
    model = Model([video], outputs)
    return model

In [10]:
tranferlearning_timedistributed_model = get_tranferlearning_timedistributed_model()

58892288/58889256 [==============================] - 5s 0us/step


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=5)`


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
tranferlearning_timedistributed_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (tranferlearning_timedistributed_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 90, 90, 3)     0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 512)           14714688  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 512)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               590592    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 15,306,565
Trainable params: 15,306,565
Non-trainable params: 0
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [14]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [15]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [16]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [17]:
tranferlearning_timedistributed_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Source path =  ./Project_data/val ; batch size = 10
./Project_data/train ; batch size = 10
Epoch 1/15


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


67/67 [==============================] - 230s 3s/step - loss: 1.5471 - categorical_accuracy: 0.3224 - val_loss: 1.2801 - val_categorical_accuracy: 0.5300

Epoch 00001: saving model to model_init_2019-12-2219_49_56.447155/model-00001-1.54707-0.32239-1.28011-0.53000.h5
Epoch 2/15
67/67 [==============================] - 121s 2s/step - loss: 1.2100 - categorical_accuracy: 0.4940 - val_loss: 0.8832 - val_categorical_accuracy: 0.6800

Epoch 00002: saving model to model_init_2019-12-2219_49_56.447155/model-00002-1.20997-0.49403-0.88318-0.68000.h5
Epoch 3/15
67/67 [==============================] - 132s 2s/step - loss: 0.9438 - categorical_accuracy: 0.6299 - val_loss: 0.7261 - val_categorical_accuracy: 0.6900

Epoch 00003: saving model to model_init_2019-12-2219_49_56.447155/model-00003-0.94378-0.62985-0.72607-0.69000.h5
Epoch 4/15
67/67 [==============================] - 115s 2s/step - loss: 0.6266 - categorical_accuracy: 0.7761 - val_loss: 0.4915 - val_categorical_accuracy: 0.7900

Epoch 00

#### Accuracy on train data obtained is 98%, whereas accuracy on validation data obtained is 89%.

#### Now we build a CNN model
### CNN

In [7]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [1,2,3,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    x = 18 #frames
    y = 90 #width of resized image
    z = 90 #height of resized image
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int( len(t) / batch_size )
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if image.shape[1] == 160:
                        # converting this image to 1:1 aspect ratio, then resizing
                        image = resize(image[:,20:140,:],(y,z)).astype(np.float32)
                    else:
                        image = resize(image,(y,z)).astype(np.float32)
                    
                    #normalising and feeding image into batch_data channel by channel
                    
                    batch_data[folder,idx,:,:,0] = [[float(i)/255 for i in row] for row in image[:,:,0]]
                    batch_data[folder,idx,:,:,1] = [[float(i)/255 for i in row] for row in image[:,:,1]]
                    batch_data[folder,idx,:,:,2] = [[float(i)/255 for i in row] for row in image[:,:,2]]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        #if there existing images not covered above
        if(len(t)%batch_size)!=0:
            #repeating the above steps for remaining images
            batch_data = np.zeros((len(t)%batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if image.shape[1] == 160:
                        # converting this image to 1:1 aspect ratio, then resizing
                        image = resize(image[:,20:140,:],(y,z)).astype(np.float32)
                    else:
                        image = resize(image,(y,z)).astype(np.float32)
                    
                    #normalising and feeding image into batch_data channel by channel
                    batch_data[folder,idx,:,:,0] = [[float(i)/255 for i in row] for row in image[:,:,0]]
                    batch_data[folder,idx,:,:,1] = [[float(i)/255 for i in row] for row in image[:,:,1]]
                    batch_data[folder,idx,:,:,2] = [[float(i)/255 for i in row] for row in image[:,:,2]]
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 60
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 60


In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [10]:
model = Sequential()
model.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,90,90,3)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [11]:
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 18, 90, 90, 32)    2624      
_________________________________________________________________
batch_normalization_1 (Batch (None, 18, 90, 90, 32)    128       
_________________________________________________________________
activation_1 (Activation)    (None, 18, 90, 90, 32)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 9, 45, 90, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 9, 45, 90, 64)     55360     
_________________________________________________________________
batch_normalization_2 (Batch (None, 9, 45, 90, 64)     256       
_________________________________________________________________
activation_2 (Activation)    (None, 9, 45, 90, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
batch_size = 10 

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [16]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 16
Source path =  Project_data/train ; batch size = 16
Epoch 1/60


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


42/42 [==============================] - 208s 5s/step - loss: 3.3108 - categorical_accuracy: 0.3625 - val_loss: 1.6683 - val_categorical_accuracy: 0.5500

Epoch 00001: saving model to model_init_2019-12-2212_33_12.571471/model-00001-3.31085-0.36350-1.66827-0.55000.h5
Epoch 2/60
42/42 [==============================] - 202s 5s/step - loss: 1.9389 - categorical_accuracy: 0.4380 - val_loss: 1.1902 - val_categorical_accuracy: 0.5600

Epoch 00002: saving model to model_init_2019-12-2212_33_12.571471/model-00002-1.89331-0.44193-1.19015-0.56000.h5
Epoch 3/60
42/42 [==============================] - 192s 5s/step - loss: 1.4991 - categorical_accuracy: 0.5076 - val_loss: 1.5731 - val_categorical_accuracy: 0.4500

Epoch 00003: saving model to model_init_2019-12-2212_33_12.571471/model-00003-1.43880-0.51433-1.57306-0.45000.h5
Epoch 4/60
42/42 [==============================] - 188s 4s/step - loss: 1.1687 - categorical_accuracy: 0.5783 - val_loss: 1.0332 - val_categorical_accuracy: 0.6500

Epoch 00

#### As we can see, we obtained a max accuracy of 82% on training data, and 77% on validation data.

### Therefore its clear that our CNN+RNN model is the better model overall with it's higher 98% training accuracy and 89% validation accuracy.